# Steps for building/testing MOM6 for GPU offload

This is following [the report from Marshall](https://github.com/marshallward/mom6-gpu-report/blob/main/gpu_report.rst#testing-in-mom6) which documents the compilation steps and testing. This notebook documents the process for Gadi.

In [1]:
cd /scratch/tm70/$USER
git clone --recursive https://github.com/NOAA-GFDL/MOM6-examples.git

fatal: destination path 'MOM6-examples' already exists and is not an empty directory.


: 128

In [2]:
module load intel-compiler/2021.10.0 netcdf/4.7.4 python3-as-python openmpi/4.1.7
cd MOM6-examples/ocean_only
make -j 4

mkdir -p build
make -C ../shared/fms \
  BUILD=/scratch/tm70/ey7514/MOM6-examples/shared/fms/build \
  CODEBASE=/scratch/tm70/ey7514/MOM6-examples/src/FMS2
cp ../src/MOM6/ac/configure.ac build/configure.ac
cp -r ../src/MOM6/ac/m4 build
cp ../src/MOM6/ac/Makefile.in build/Makefile.in
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
mkdir -p /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
cp ../../src/MOM6/ac/deps/Makefile.fms.in /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/Makefile.in
cp ../../src/MOM6/ac/deps/configure.fms.ac /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/configure.ac
cp -r ../../src/MOM6/ac/deps/m4 /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
autoreconf build
autoreconf /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
cd /scratch/tm70/ey7514/MOM6-examples/shared/fms/build && \
  PATH="/apps/openmpi/wrapper/fortran:/apps/openmpi/wrapper:/apps/openmpi/4.1.7/bin:/apps/netcdf/4.7.4/bin:/apps/intel-ct/wrapper:/apps/intel

In [3]:
cd double_gyre
mpiexec -n 1 ../build/MOM6

NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_IO_NML
 HEADER_BUFFER_VAL       =       16384,
 GLOBAL_FIELD_ON_ROOT_PE = T,
 IO_CLOCKS_ON    = F,
 SHUFFLE =           0,
 DEFLATE_LEVEL   =          -1,
 CF_COMPLIANCE   = F
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =   44
  Y-AXIS =   40
 MOM_inc domain decomposition
whalo =    2, ehalo =    2, shalo =    2, nhalo =    2
  X-AXIS =   22
  Y-AXIS =   20
NOTE: diag_manager_mod::diag_manager_init: diag_manager is using fms2_io
NOTE: diag_manager_mod::diag_manager_init: prepend_date only supported when diag_manager_init is called with time_init present.
 MOM_in domain decompos

Things ran ok, let's profile for hotspots.

In [4]:
module load intel-vtune

In [6]:
rm -rf mom6-prof-vtune*
mpiexec -n 1 vtune -collect hotspots -knob sampling-mode=hw -knob enable-stack-collection=true -r mom6-prof-vtune ../build/MOM6

vtune: Analyzing data in the node-wide mode. The hostname (gadi-login-09.gadi.nci.org.au) will be added to the result path/name.
vtune: Warning: Access to /proc/kallsyms file is limited. Consider changing /proc/sys/kernel/kptr_restrict to 0 to enable resolution of OS kernel and kernel module symbols.
vtune: Warning: To analyze modules at the kernel level in this configuration, load the Intel sampling driver and set an unlimited (0) value for the Stack size option (if you require stack collection). Alternatively, enable access to kernel-mode monitoring by setting the /proc/sys/kernel/perf_event_paranoid value to 1 or lower.
vtune: Warning: To profile kernel modules during the session, make sure they are available in the /lib/modules/kernel_version/ location.
vtune: Warning: Consider reducing possible collection overhead by setting the /proc/sys/kernel/perf_event_paranoid value to 0 (or less).
vtune: Collection started.
NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_

: 134

In [11]:
vtune -report=hotspots -r mom6-prof-vtune.gadi-login-??.gadi.nci.org.au -format=csv | column -ts $'\t' | grep MOM6

vtune: Using result path `/scratch/tm70/ey7514/MOM6-examples/ocean_only/double_gyre/mom6-prof-vtune.gadi-login-09.gadi.nci.org.au'
vtune: Executing actions 100 % done                                            
btstep                            1.002349  1.002349                 0.0                 0.0                     10643000000           91.3                        91.3                                                0.320163                          MOM6                 btstep                            MOM_barotropic.F90               0x61dd80
vertvisc_coef                     0.355834  0.355834                 0.0                 0.0                     2856500000            92.9                        92.9                                                0.365482                          MOM6                 vertvisc_coef                     MOM_vert_friction.F90            0x10b2120
meridional_flux_adjust            0.205482  0.205482                 0.0                 0.0    

In [12]:
vtune -report=hotspots -r mom6-prof-vtune.gadi-login-??.gadi.nci.org.au -format=csv -group-by=source-file-path | column -ts $'\t' | grep MOM6

vtune: Using result path `/scratch/tm70/ey7514/MOM6-examples/ocean_only/double_gyre/mom6-prof-vtune.gadi-login-09.gadi.nci.org.au'
vtune: Executing actions 100 % done                                            
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_continuity_PPM.F90                                                                  1.207831  1.207831                 0.0                 0.0                     10730000000           68.7                        68.7                                                0.385135
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_barotropic.F90                                                                      1.182772  1.182772                 0.0                 0.0                     12049500000           86.0                        86.0                                                0.345367
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/parameterizations/vertical/MOM_vert_friction.F90                               

I'm not confident that I'm using VTune correctly, but it looks like MOM_barotropic is an expensive step.

But looks like `core/MOM_barotropic.F90:btstep` is the most time-consuming function.

Turns out it's a 2.5k line subroutine: https://github.com/NOAA-GFDL/MOM6/blob/dev/gfdl/src/core/MOM_barotropic.F90#L2848

Otherwise some of the subroutines in `core/MOM_continuity_PPM.F90` like [`zonal_flux_adjust`](https://github.com/NOAA-GFDL/MOM6/blob/dev/gfdl/src/core/MOM_continuity_PPM.F90#L1094) or [`meridional_flux_adjust`](https://github.com/NOAA-GFDL/MOM6/blob/dev/gfdl/src/core/MOM_continuity_PPM.F90#L1995) look like more manageable candidates.

Let's see if the `benchmark` test case looks any different.

In [31]:
cd ../benchmark

In [32]:
rm -rf mom6-prof-vtune*
mpiexec -n 1 ../build/MOM6

NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_IO_NML
 HEADER_BUFFER_VAL       =       16384,
 GLOBAL_FIELD_ON_ROOT_PE = T,
 IO_CLOCKS_ON    = F,
 SHUFFLE =           0,
 DEFLATE_LEVEL   =          -1,
 CF_COMPLIANCE   = F
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to   955296.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =  360
  Y-AXIS =  180
 MOM_inc domain decomposition
whalo =    2, ehalo =    2, shalo =    2, nhalo =    2
  X-AXIS =  180
  Y-AXIS =   90
NOTE: diag_manager_mod::diag_manager_init: diag_manager is using fms2_io
NOTE: diag_manager_mod::diag_manager_init: prepend_date only supported when diag_manager_

: 139

Weird... Maybe it's a problem with the intel compiler? Let's try build again...

I spent an afternoong looking at this with no luck.

Mighang told me it's because of the OOM killer on the login nodes, so I needed to run the test in a job. I'm used to getting a more informative message from cgroups, but idk how it's setup on Gadi...

Anyway, looking at the results of the vtune report:

In [1]:
# job submission script:
cd /scratch/tm70/$USER/MOM6-examples/ocean_only/benchmark
echo '#!/bin/bash
#PBS -N MOM6_test
#PBS -P tm70
#PBS -q normal
#PBS -l walltime=02:00:00
#PBS -l ncpus=48
#PBS -l mem=192GB
#PBS -l jobfs=10GB
#PBS -l storage=scratch/tm70
#PBS -l wd

module purge
module load netcdf intel-compiler openmpi intel-vtune

MOM6_EXEC=../build/MOM6

mpirun -np $PBS_NCPUS vtune -collect hotspots -r mom6-prof-vtune $MOM6_EXEC > mom6.out 2> mom6.err' > submit.sh

qsub submit.sh

133140840.gadi-pbs


In [2]:
# per subroutine
# generated with 
# vtune -report=hotspots -r mom6-prof-vtune.gadi-cpu-clx-1883.gadi.nci.org.au/ -format=csv > prof.csv
head prof.csv | column -ts $'\t'

Function                         CPU Time   CPU Time:Effective Time  CPU Time:Spin Time  CPU Time:Overhead Time  Module       Function (Full)                  Source File                 Start Address
btstep                           54.262846  54.262846                0.0                 0.0                     MOM6         btstep                           MOM_barotropic.F90          0x61b3f0
uct_mm_iface_progress            49.399108  49.399108                0.0                 0.0                     libuct.so.0  uct_mm_iface_progress            mm_iface.c                  0x18260
uct_mm_iface_poll_fifo           43.640615  43.640615                0.0                 0.0                     libuct.so.0  uct_mm_iface_poll_fifo           mm_iface.c                  0x18262
ucp_worker_progress              42.150527  42.150527                0.0                 0.0                     libucp.so.0  ucp_worker_progress              ucp_worker.c                0x48370
vertvisc_coef     

In [3]:
# grouped by source
# generated with
# vtune -report=hotspots -r mom6-prof-vtune.gadi-cpu-clx-1883.gadi.nci.org.au/ -format=csv  -group-by=source-file-path > prof-grouped.csv
head prof-grouped.csv | column -ts $'\t'

Source File Path                                                                                               CPU Time    CPU Time:Effective Time  CPU Time:Spin Time  CPU Time:Overhead Time
/jobfs/126523667.gadi-pbs/0/ucx/1.17.0/build/src/uct/../../../source/ucx-1.17.0/src/uct/sm/mm/base/mm_iface.c  145.993677  145.993677               0.0                 0.0
[Unknown]                                                                                                      108.672162  104.853139               3.819023            0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_continuity_PPM.F90                                    77.840111   77.840111                0.0                 0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_barotropic.F90                                        62.977219   62.977219                0.0                 0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/parameterizations/vertical/MOM_vert_friction.F90               54.716628 

Ignoring the UCX and `[Unknown]`, looks like the outcome is roughly the same where `MOM_baratropic.F90:btstep` takes the longest, followed by smaller subroutines in `MOM_continuity_PPM.F90`.

## Comparing results of `double_gyre` serial with Marshall's OpenMP changes

In [5]:
cd /scratch/tm70/$USER/MOM6-examples/src/MOM6

In [7]:
git remote add -t pgf_gpu_omp marshall-omp https://github.com/marshallward/MOM6
git remote -v

error: remote marshall-omp already exists.
marshall-omp	https://github.com/marshallward/MOM6 (fetch)
marshall-omp	https://github.com/marshallward/MOM6 (push)
origin	https://github.com/NOAA-GFDL/MOM6.git (fetch)
origin	https://github.com/NOAA-GFDL/MOM6.git (push)


In [10]:
git pull marshall-omp

Auto-merging src/core/MOM.F90
error: Terminal is dumb, but EDITOR unset
Not committing merge; use 'git commit' to complete the merge.


: 1

So from here, I tried to load the `nvhpc-compilers` module, but that module doesn't seem to like the bash kernel setup.

In [12]:
cd /scratch/tm70/$USER/MOM6-examples/ocean_only
module purge
export NCI_NVIDIA_HPC_SDK_NO_WARN=1 # needed to load nvhpc-compilers in with bash-kernel
module load pbs nvhpc-compilers openmpi netcdf python3-as-python

In [14]:
make clean
FCFLAGS="-g -O0 -mp=gpu -Minfo=mp" make

rm -rf build
make -C ../shared/fms clean
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
rm -rf build
make[1]: Leaving directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
mkdir -p build
cp ../src/MOM6/ac/configure.ac build/configure.ac
cp -r ../src/MOM6/ac/m4 build
autoreconf build
make -C ../shared/fms \
  BUILD=/scratch/tm70/ey7514/MOM6-examples/shared/fms/build \
  CODEBASE=/scratch/tm70/ey7514/MOM6-examples/src/FMS2
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
mkdir -p /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
cp ../../src/MOM6/ac/deps/Makefile.fms.in /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/Makefile.in
cp ../../src/MOM6/ac/deps/configure.fms.ac /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/configure.ac
cp -r ../../src/MOM6/ac/deps/m4 /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
autoreconf /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
cd /scratch/tm70/ey7514/MOM6-examples/shared

: 2

Ok... doesn't look like NCI's NetCDF work with the NVHPC compilers.

I installed NetCDF with NVHPC and Spack. Needed to point spack to the NVHPC compilers, and then install netcdf e.g. `spack install --reuse -v netcdf-fortran%nvhpc`. Turns out there are some configs preventing rebuild of many packages (hooray).

Continuing on after installing NetCDF with NVHPC compilers with Spack.

In [15]:
. /g/data/tm70/$USER/spack/0.22/spack-config/spack-enable.bash
spack load netcdf-fortran%nvhpc

Using configuration in spack/etc/spack


In [43]:
cd /scratch/tm70/$USER/MOM6-examples/ocean_only
make clean
# -mp=gpu is for gpu offloading of openmp/openacc directives
FCFLAGS="-g -O0 -mp=gpu -Minfo=mp" LDFLAGS="-mp=gpu" make -j

rm -rf build
make -C ../shared/fms clean
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
rm -rf build
make[1]: Leaving directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
mkdir -p build
make -C ../shared/fms \
  BUILD=/scratch/tm70/ey7514/MOM6-examples/shared/fms/build \
  CODEBASE=/scratch/tm70/ey7514/MOM6-examples/src/FMS2
cp ../src/MOM6/ac/configure.ac build/configure.ac
cp -r ../src/MOM6/ac/m4 build
cp ../src/MOM6/ac/Makefile.in build/Makefile.in
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
mkdir -p /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
cp ../../src/MOM6/ac/deps/Makefile.fms.in /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/Makefile.in
cp ../../src/MOM6/ac/deps/configure.fms.ac /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/configure.ac
cp -r ../../src/MOM6/ac/deps/m4 /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
autoreconf /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
autoreconf bu

Looks like there are some existing openmp directives implemented throughout `mpp` (dunno what that is) and `fms`. from `-Minfo=mp` output, the `target` directives indcate the GPU kernel code is being generated, but the directives without target don't say the same. I can speculate that `!$omp target ...` is probably necessary for GPU offloading. I could be wrong though.

Something else I notice from the `Minfo` is the large number of `Generating implicit map...` lines. Might mean there are more data transfers happening than Marshall is aware of. Probably not a problem if an `enter data` statement is placed early on. But better to be explicit with the data transfers.

Build looks ok, let's check output of old stats file

In [42]:
module unload netcdf

In [21]:
mv double_gyre/ocean.stats{,intel-serial}
sha256sum double_gyre/ocean.stats.intel-serial
cat !$

mv: cannot stat 'double_gyre/ocean.stats': No such file or directory
82acf384bd030b4e4a62de5ea7b807d6ac3cb5b0c9c03ff340efda4f24c651ce  double_gyre/ocean.stats.intel-serial
cat double_gyre/ocean.stats.intel-serial
  Step,       Day,  Truncs,      Energy/Mass,      Maximum CFL,  Mean sea level,   Total Mass,    Frac Mass Err
            [days]                 [m2 s-2]           [Nondim]        [m]             [kg]           [Nondim]
     0,       0.000,     0, En 1.4237466946781553E-13, CFL  0.00000, SL  3.8085E-12, Mass 5.28818E+18, Me  0.00E+00
    72,       1.000,     0, En 4.5804317886464199E-06, CFL  0.00023, SL  3.8085E-12, Mass 5.28818E+18, Me -1.70E-17
   144,       2.000,     0, En 8.9582059493820867E-06, CFL  0.00040, SL  3.8085E-12, Mass 5.28818E+18, Me -9.21E-19
   216,       3.000,     0, En 1.1667077649373802E-05, CFL  0.00045, SL  3.8085E-12, Mass 5.28818E+18, Me  7.15E-18
   288,       4.000,     0, En 1.5422955375278674E-05, CFL  0.00044, SL  3.8085E-12, Mass 5.28818E+18

In [32]:
cd /scratch/tm70/$USER/MOM6-examples/ocean_only/double_gyre

In [37]:
echo '#!/bin/bash
#PBS -N MOM6_test_omp
#PBS -P tm70
#PBS -q gpuvolta
#PBS -l walltime=02:00:00
#PBS -l ncpus=12
#PBS -l mem=20GB
#PBS -l ngpus=1
#PBS -l jobfs=10GB
#PBS -l storage=scratch/tm70+gdata/tm70
#PBS -l wd

pwd

module purge
module load nvhpc-profilers openmpi
. /g/data/tm70/$USER/spack/0.22/spack-config/spack-enable.bash

MOM6_EXEC=../build/MOM6

export LD_LIBRARY_PATH=$(spack location -i netcdf-fortran%nvhpc)/lib:$(spack location -i netcdf-c)/lib:$LD_LIBRARY_PATH
mpirun -np 1 nsys profile --stats=true -o mom6-omp-profile.out $MOM6_EXEC > mom6.out 2> mom6.err' > submit.sh


In [38]:
qsub submit.sh

133146209.gadi-pbs


```
Generating '/jobfs/133151831.gadi-pbs/nsys-report-d471.qdstrm'
[1/8] [========================100%] mom6-omp-profile.out.nsys-rep
[2/8] [========================100%] mom6-omp-profile.out.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /scratch/tm70/ey7514/MOM6-examples/ocean_only/double_gyre/mom6-omp-profile.out.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)     Med (ns)    Min (ns)    Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  -----------  -----------  ---------  -----------  ------------  ----------------------
     57.8      53729233503       1731   31039418.5   10132304.0       1575  19318104961   464706560.2  poll                  
     41.1      38265724629         72  531468397.6      97244.0       1037  18715209030  3095366665.1  epoll_wait            
      0.7        637820016          2  318910008.0  318910008.0  225841004    411979012   131619447.7  sem_wait              
      0.3        236377237        975     242438.2      28775.0       1012     35534639     1800427.8  ioctl                 
      0.1         61069236         13    4697633.5      68397.0      59149     46570326    13110260.9  pthread_cond_wait     
      0.0         23486500        200     117432.5      20213.0       1052      8022951      645163.1  write                 
      0.0         10196560        603      16909.7      15421.0       1073        48215        9499.7  fgets                 
      0.0          8001038        132      60613.9       7408.0       1359       673366      134037.2  fopen                 
      0.0          6272159        170      36895.1       6092.0       1175       841979      133823.7  open                  
      0.0          6241314         17     367136.1      57141.0      21771      3926901      970149.5  pthread_mutex_lock    
      0.0          5070754          1    5070754.0    5070754.0    5070754      5070754           0.0  nanosleep             
      0.0          3341347        100      33413.5       3852.5       1073       208810       52794.5  fclose                
      0.0          2515825        350       7188.1       4539.5       1004       242559       18972.2  read                  
      0.0          2155848        118      18269.9      12255.5       9134        67525       13374.0  fputc                 
      0.0          1906186         35      54462.5       7031.0       5136      1489901      250080.0  mmap64                
      0.0          1550059          2     775029.5     775029.5     690148       859911      120040.6  waitpid               
      0.0          1455667         84      17329.4      15172.0       1100       223413       25190.9  fwrite                
      0.0          1343598          4     335899.5     327944.5     274934       412775       57495.0  ftruncate             
      0.0          1215098          5     243019.6     218897.0     165346       359682       72462.0  pthread_create        
      0.0           924978          5     184995.6     188134.0     168092       196040       11793.0  usleep                
      0.0           880028          2     440014.0     440014.0     309049       570979      185212.5  posix_spawn           
      0.0           808204         16      50512.8      29594.0       1548       461745      110302.1  pthread_cond_broadcast
      0.0           530752          3     176917.3       6920.0       4718       519114      296353.1  pthread_join          
      0.0           477563        164       2912.0       2610.0       1007        17499        2202.6  socket                
      0.0           387744          2     193872.0     193872.0       1481       386263      272082.0  recv                  
      0.0           280486         31       9047.9       7927.0       1754        17947        5244.1  sendmsg               
      0.0           240855         14      17203.9       7298.0       1619        65198       21681.4  fread                 
      0.0           240307         35       6865.9       2685.0       1092        41483       10300.3  fflush                
      0.0           223684         11      20334.9      13368.0       2588        64769       17847.0  writev                
      0.0           204046         28       7287.4       5599.0       1895        17314        3888.4  fgetc                 
      0.0           204019         50       4080.4       3427.5       1105        10090        1862.6  open64                
      0.0           128075         57       2246.9       1951.0       1001         5444        1202.4  epoll_ctl             
      0.0           126335         36       3509.3       2495.5       1729        14518        2577.1  mprotect              
      0.0           107371         40       2684.3       1998.5       1003         7186        1507.0  recvmsg               
      0.0            96450          5      19290.0      19992.0      14009        23640        3668.4  openat                
      0.0            54907         10       5490.7       4195.0       2747        11008        2946.2  mmap                  
      0.0            48878         28       1745.6       1398.0       1002         3902         823.5  bind                  
      0.0            48159          5       9631.8       7906.0       1702        24389        8700.8  shmget                
      0.0            46408          2      23204.0      23204.0       2789        43619       28871.2  shutdown              
      0.0            31800          1      31800.0      31800.0      31800        31800           0.0  connect               
      0.0            28345          3       9448.3       8376.0       6772        13197        3344.0  shmat                 
      0.0            26631          3       8877.0       7609.0       6329        12693        3366.2  socketpair            
      0.0            25025         19       1317.1       1303.0       1036         1680         177.0  fcntl                 
      0.0            24321          4       6080.3       6545.5       4334         6896        1178.8  process_vm_writev     
      0.0            23170          3       7723.3       8041.0       6599         8530        1003.9  pipe                  
      0.0            13060          1      13060.0      13060.0      13060        13060           0.0  shmdt                 
      0.0             7938          2       3969.0       3969.0       3295         4643         953.2  pthread_spin_trylock  
      0.0             7141          4       1785.3       1485.5       1029         3141         930.9  shmctl                
      0.0             6583          1       6583.0       6583.0       6583         6583           0.0  send                  
      0.0             5805          2       2902.5       2902.5       1578         4227        1873.1  process_vm_readv      
      0.0             4937          1       4937.0       4937.0       4937         4937           0.0  pipe2                 
      0.0             4693          2       2346.5       2346.5       1074         3619        1799.6  recvfrom              
      0.0             4682          3       1560.7       1608.0       1354         1720         187.5  listen                
      0.0             1498          1       1498.0       1498.0       1498         1498           0.0  fopen64               

[5/8] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)          Name        
 --------  ---------------  ---------  ----------  ----------  --------  --------  -----------  --------------------
     67.4        748628124      14408     51959.2      9405.0      1225  19935812     219540.7  cuStreamSynchronize 
     18.2        201712784       7920     25468.8     17447.5     13373   1237066      27875.0  cuMemcpyDtoHAsync_v2
      6.7         73958281      12255      6034.9      5954.0      2195    989135       9823.3  cuMemcpyHtoDAsync_v2
      3.4         37616020       7920      4749.5      4003.0      3059    152991       2924.2  cuLaunchKernel      
      2.2         24464047          1  24464047.0  24464047.0  24464047  24464047          0.0  cuMemHostAlloc      
      1.9         21278185          1  21278185.0  21278185.0  21278185  21278185          0.0  cuMemAllocManaged   
      0.2          2003728          1   2003728.0   2003728.0   2003728   2003728          0.0  cuModuleLoadDataEx  
      0.1           909395          1    909395.0    909395.0    909395    909395          0.0  cuMemAllocHost_v2   
      0.0           442050         31     14259.7      6869.0      2395    222260      38771.1  cuMemAlloc_v2       
      0.0             4775          2      2387.5      2387.5      2046      2729        483.0  cuInit              
      0.0             3751          4       937.8       435.5       180      2700       1181.0  cuCtxSetCurrent     

[6/8] Executing 'cuda_gpu_kern_sum' stats report

 Time (%)  Total Time (ns)  Instances  Avg (ns)  Med (ns)  Min (ns)  Max (ns)  StdDev (ns)                                Name                               
 --------  ---------------  ---------  --------  --------  --------  --------  -----------  -----------------------------------------------------------------
     62.2        472563843        720  656338.7  656381.0    655357    658878        482.5  nvkernel_mom_pressureforce_mont_set_pbce_bouss__F1L761_46_       
      9.3         70414859        720   97798.4   97792.0     97759     98304         29.8  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1223_38_
      7.3         55169202        720   76623.9   76639.0     76543     77760         61.4  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1693_53_
      5.0         38007019        720   52787.5   52799.0     52736     52928         22.3  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1788_61_
      4.5         33810559        720   46959.1   46944.0     46911     48032         45.5  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1484_51_
      3.5         26749556        720   37152.2   37152.0     37088     37983         48.6  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1251_40_
      2.4         18036126        720   25050.2   25056.0     24991     26048         41.7  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1097_32_
      2.1         15963692        720   22171.8   22176.0     22143     22688         25.8  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1472_49_
      1.4         10719328        720   14888.0   14880.0     14848     15008         15.7  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1823_63_
      1.2          9481496        720   13168.7   13152.0     13151     13281         18.8  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1780_59_
      1.2          8847708        720   12288.5   12288.0     12255     12736         22.1  nvkernel_mom_pressureforce_fv_pressureforce_fv_bouss__F1L1037_26_

[7/8] Executing 'cuda_gpu_mem_time_sum' stats report

 Time (%)  Total Time (ns)  Count  Avg (ns)  Med (ns)  Min (ns)  Max (ns)  StdDev (ns)           Operation          
 --------  ---------------  -----  --------  --------  --------  --------  -----------  ----------------------------
     59.8         32537814  12255    2655.1    3520.0      1343      6143       1319.0  [CUDA memcpy Host-to-Device]
     40.2         21849213   7920    2758.7    2336.0      2271      4320        703.4  [CUDA memcpy Device-to-Host]

[8/8] Executing 'cuda_gpu_mem_size_sum' stats report

 Total (MB)  Count  Avg (MB)  Med (MB)  Min (MB)  Max (MB)  StdDev (MB)           Operation          
 ----------  -----  --------  --------  --------  --------  -----------  ----------------------------
    202.429   7920     0.026     0.020     0.020     0.041        0.009  [CUDA memcpy Device-to-Host]
    144.941  12255     0.012     0.020     0.000     0.041        0.012  [CUDA memcpy Host-to-Device]

Generated:
    /scratch/tm70/ey7514/MOM6-examples/ocean_only/double_gyre/mom6-omp-profile.out.nsys-rep
    /scratch/tm70/ey7514/MOM6-examples/ocean_only/double_gyre/mom6-omp-profile.out.sqlite
```

## Confirming similarity of results

In [36]:
# build in serial
cd /scratch/tm70/$USER/MOM6-examples/ocean_only
make clean
. /g/data/tm70/$USER/spack/0.22/spack-config/spack-enable.bash
spack load netcdf-fortran%nvhpc
export NCI_NVIDIA_HPC_SDK_NO_WARN=1 # needed to load nvhpc-compilers in with bash-kernel
module load nvhpc-compilers/24.7 openmpi python3-as-python
FCFLAGS="-g -O0 -Minfo=all -Mnofma" make -j

rm -rf build
make -C ../shared/fms clean
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
rm -rf build
make[1]: Leaving directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
Using configuration in spack/etc/spack
mkdir -p build
make -C ../shared/fms \
  BUILD=/scratch/tm70/ey7514/MOM6-examples/shared/fms/build \
  CODEBASE=/scratch/tm70/ey7514/MOM6-examples/src/FMS2
cp ../src/MOM6/ac/configure.ac build/configure.ac
cp -r ../src/MOM6/ac/m4 build
cp ../src/MOM6/ac/Makefile.in build/Makefile.in
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
mkdir -p /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
cp ../../src/MOM6/ac/deps/Makefile.fms.in /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/Makefile.in
cp ../../src/MOM6/ac/deps/configure.fms.ac /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/configure.ac
cp -r ../../src/MOM6/ac/deps/m4 /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
autoreconf build
autoreconf /scratch/

In [37]:
cd /scratch/tm70/$USER/MOM6-examples/ocean_only/double_gyre
export LD_LIBRARY_PATH=$(spack location -i netcdf-fortran%nvhpc)/lib:$(spack location -i netcdf-c)/lib:$LD_LIBRARY_PATH
mpiexec -n 1 ../build/MOM6

NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_IO_NML
 HEADER_BUFFER_VAL =        16384,
 GLOBAL_FIELD_ON_ROOT_PE =  T,
 IO_CLOCKS_ON =  F,
 SHUFFLE =            0,
 DEFLATE_LEVEL =           -1,
 CF_COMPLIANCE =  F
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =   44
  Y-AXIS =   40
 MOM_inc domain decomposition
whalo =    2, ehalo =    2, shalo =    2, nhalo =    2
  X-AXIS =   22
  Y-AXIS =   20
NOTE: diag_manager_mod::diag_manager_init: diag_manager is using fms2_io
NOTE: diag_manager_mod::diag_manager_init: prepend_date only supported when diag_manager_init is called with time_init present.
 MOM_in domain decomposition
w

In [38]:
# save results
mv ocean.stats{,.nvfortran-serial}

In [39]:
# rebuild for gpu offload
cd /scratch/tm70/$USER/MOM6-examples/ocean_only
make clean
FCFLAGS="-g -O0 -mp=gpu -Minfo=mp -Mnofma" LDFLAGS="-mp=gpu" make -j

rm -rf build
make -C ../shared/fms clean
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
rm -rf build
make[1]: Leaving directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
mkdir -p build
make -C ../shared/fms \
  BUILD=/scratch/tm70/ey7514/MOM6-examples/shared/fms/build \
  CODEBASE=/scratch/tm70/ey7514/MOM6-examples/src/FMS2
cp ../src/MOM6/ac/configure.ac build/configure.ac
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
mkdir -p /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
cp -r ../src/MOM6/ac/m4 build
cp ../src/MOM6/ac/Makefile.in build/Makefile.in
cp ../../src/MOM6/ac/deps/Makefile.fms.in /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/Makefile.in
cp ../../src/MOM6/ac/deps/configure.fms.ac /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/configure.ac
cp -r ../../src/MOM6/ac/deps/m4 /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
autoreconf /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
autoreconf bu

In [40]:
cd /scratch/tm70/$USER/MOM6-examples/ocean_only/double_gyre

In [30]:
echo '#!/bin/bash
#PBS -N MOM6_test_omp
#PBS -P tm70
#PBS -q gpuvolta
#PBS -l walltime=02:00:00
#PBS -l ncpus=12
#PBS -l mem=20GB
#PBS -l ngpus=1
#PBS -l jobfs=10GB
#PBS -l storage=scratch/tm70+gdata/tm70
#PBS -l wd

pwd

module purge
module load openmpi
. /g/data/tm70/$USER/spack/0.22/spack-config/spack-enable.bash

MOM6_EXEC=../build/MOM6

export LD_LIBRARY_PATH=$(spack location -i netcdf-fortran%nvhpc)/lib:$(spack location -i netcdf-c)/lib:$LD_LIBRARY_PATH
mpirun -np 1 $MOM6_EXEC > mom6.out 2> mom6.err' > submit.sh

In [41]:
qsub submit.sh

133171888.gadi-pbs


In [42]:
git diff --color --minimal --word-diff --no-index -- ocean.stats{,.nvfortran-serial}

Seems like there's bit-wise reproducibility between serial and GPU offloading (requires FMA to be turned off)!